In [1]:
from nltk.corpus import stopwords
import pandas as pd
import re
import pickle

In [2]:
mycorpus=list()
df=pd.read_csv('../data/training_data.tsv',delimiter='\t')
df=df.dropna()
y=df['label']
X=df['sent']
X_list=list()
for _ in X:
    t=re.findall('[a-z]+',_.lower())
    t=[word for word in t]
    X_list+=t
X_list=set(X_list)
y_list=list()
for _ in y:
    t=re.findall('[a-z]+',_.lower())
    t=[word for word in t]
    y_list+=t
y_list=set(y_list)

In [3]:
mycorpus+=list(X_list-y_list)
cfile=open("mycorpus.pickle",'wb')
pickle.dump(mycorpus,cfile)
cfile.close()

#Saving mycorpus in file
file = open('mycorpus.txt','w')
for _ in mycorpus:
    file.write(_+'\n')
file.close()

In [7]:
import pandas as pd
import numpy as np
from sklearn import svm
import re
import pickle
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [10]:
#getting data
df=pd.read_csv("../data/training_data.tsv",delimiter='\t')
df=df.dropna()
y=df['label']
X=df['sent']

#Processing Data of X
x=list()
for _ in X:
    t=re.findall('[a-z]+',_.lower())
    t=[word for word in t if not word in set(stopwords.words('english'))]
    x.append(" ".join(t))
x=pd.Series(x)

#processing data of y
Y=list()
for _ in y:
    if _ =="Not Found":
        Y.append("Not Found")
    else:
        Y.append("Found")
y=pd.Series(Y)  

#splitting data for train and test
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

#Making list of x
x_train_list=list()
for i in X_train:
    x_train_list.append(i)
x_test_list=list()
for j in X_test:
    x_test_list.append(j)

#Vectorizing x
vectorizer = CountVectorizer(max_features=1000) # n-grams Bag of word
train_data = vectorizer.fit_transform(x_train_list) # expects a list of strings
np.asarray(train_data)
test_data = vectorizer.fit_transform(x_test_list) # expects a list of strings
np.asarray(test_data)

#Modelling
#split=train_data,test_data,y_train,y_test
clf=svm.LinearSVC()
clf=clf.fit(train_data,y_train)

#Saving Model in pickle ("model.pickle")
f=open("model.pickle",'wb')
pickle.dump(clf, f)
f.close()

#Checking Accuracy
accuracy=clf.score(test_data,y_test)

#Predicting result
result=clf.predict(test_data)
result_list=list(result)

#creating confusion matrix
cm=confusion_matrix(y_test,result)

# saving file of x_test_list,y_test_list,result_list
y_test_list=list(y_test)
file=open("y_test_result.tsv",'w')
file.write("sent\tlabel\ttest\n")
for i in range(len(y_test)):
    file.write(x_test_list[i]+'\t'+y_test_list[i]+'\t'+result_list[i]+'\n')

In [13]:
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer

#Loading trained model
f=open("model.pickle",'rb')
clf=pickle.load(f)

#Loading eval file
eval_data=open("../data/eval_data.txt",'r')
X=list()
all_words=list()
for lines in eval_data:
    all_words+=(lines.strip('\n').split(' '))
    X.append(lines.strip('\n'))
eval_data.close()
all_words=list(set(all_words))

#Vectorizing and converting to array: X_test
vectorizer=CountVectorizer(max_features=1000)
X_test=vectorizer.fit_transform(X).toarray()

#Predicting Result (found/not found)
result=list(clf.predict(X_test))

#Setting mycopus.txt
mycorpus=['i','me','at','to','date','time','for','tommorow','tonight','today',\
           'sunday','monday','tuesday','wednesday','thursday','friday','saturday',\
           'morning','evening']
#For found, extract result text
def extract(index):
    #Create the extractor function
    temp=X[index].lower()
    t=re.findall('remi[a-z]+ me? to? (.+?) at',temp)
    if len(t)==0:
        t=re.findall('rem[a-z]+ me? to? (.+?) on',temp)
    if len(t)==0:
        t=temp
        t+=' .y'
        t=re.findall('remi[\w.]+ (.+?) {}'.format(t.split(" ")[len(t.split(" "))-1]),t)
    if len(t)>0:
        t=str(t[0]).split()    
        t=[word for word in t if not word in mycorpus]
    if len(t)>0:
        result[index]=" ".join(t)
    else:
        result[index]="Not Found"

for i in range(len(X)):
    if(result[i]=="Found"):
        extract(i)
        
#----Saving result------------
file=open("Result.csv",'w')
file.write("sent\tlabel\n")
for i in range(len(result)):
    file.write(str(X[i])+'\t'+str(result[i])+'\n')
file.close()